# Guided Project: Building Fast Queries on a CSV
Dataset: https://www.kaggle.com/ionaskel/laptop-prices

Background:
 - Take the perspective as the owner of an online laptop store
 - Want to build a way to answer business questions about inventory

Objectives:
 - Create a class that represents our inventory.
 - Methods in that class will implement the queries to answer questions about inventory
 - Pre-process the inventory data to make queries run faster
 
Queries to Answer:
  - Given a laptop ID, find the corresponding data
  - Given an amount of money, find whether there are two laptops whose total price is that given amount
  - Identify all laptops whose price falls within a given budget

# Step 1: Reading the Inventory
1. Use the csv module to read the 'laptops.csv' file
2. Separate the header row
3. Separate the remaining rows

In [1]:
# open 'laptops.csv' file and convert to list of lists

import csv

with open('laptops.csv', encoding='utf8') as file:
    convert_file_to_list = list(csv.reader(file))
    header = convert_file_to_list[0]
    rows = convert_file_to_list[1:]
    print(header)
    print('')
    print(rows[1:5])
    

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']

[['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'], ['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB SSD', 'Intel Iris Plus Graphics 650', 'macOS', '1.37kg', '1803']]


# Step 2: Create the Inventory Class

In [2]:
# 1st version of Inventory class
# implement the inventory class constructor
# takes name of the CSV file as argument, then read rows

class Inventory():
    def __init__(self, csv_filename):
        
        # open file and convert to list of lists
        with open(csv_filename, encoding='utf8') as file:
            convert_file_to_list = list(csv.reader(file))
            self.header = convert_file_to_list[0]
            self.rows = convert_file_to_list[1:]
            
            # convert price column from string to integer
            for row in self.rows:
                row[12] = int(row[12])
                
# create instance/instantiate Inventory class object     
test = Inventory('laptops.csv')
print(len(test.rows))

1303


# Step 3: Update Inventory Class, 1st Update - Laptop ID Look-Up
 - Implement a method to find a laptop within the inventory by searching for the ID

In [3]:
# 2nd version of Inventory class - add get_laptop_from_id() method
# implement way to look up laptop from a given identifier
# if customer comes to store with receipt, can find laptop with identifier code

class Inventory():
    def __init__(self, csv_filename):
        
        # open file and convert to list of lists
        with open(csv_filename, encoding='utf8') as file:
            convert_file_to_list = list(csv.reader(file))
            self.header = convert_file_to_list[0]
            self.rows = convert_file_to_list[1:]
            
            # convert price column from string to integer
            for row in self.rows:
                row[12] = int(row[12])
                
    # take a laptop id and return corresponding info if match found
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row

test = Inventory('laptops.csv')
print(test.get_laptop_from_id('3362737'))
print(test.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


# Step 4: Inventory Class, 2nd Update - ID Look-Up Method Time Complexity Improvement
 - add data pre-processing to improve the time complexity of the ID look-up method by playing data into a dictionary

In [4]:
# 3rd version of Inventory class - add data pre-processing to __init__()
# improve ID lookup function time complexity from O(R) to O(1) by
# pre-processing data from list to dictionary
# dictionary is chosen over set as need to also retrieve remaining row info

class Inventory():
    def __init__(self, csv_filename):
        
        # open file and convert to list of lists
        with open(csv_filename, encoding='utf8') as file:
            convert_file_to_list = list(csv.reader(file))
            self.header = convert_file_to_list[0]
            self.rows = convert_file_to_list[1:]
            
            # convert price column from string to integer
            for row in self.rows:
                row[12] = int(row[12])
        
        # pre-process data into dictionary
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
                
    # take a laptop id and return corresponding info if match found
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
    
    # new faster search method   
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]

test = Inventory('laptops.csv')
print(test.get_laptop_from_id_fast('3362737'))
print(test.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


# Step 5: Measure performance of the two ID look-up methods

In [5]:
# measure execution times of get_laptop_from_id and get_laptop_from_id_fast 

import time, random

# random list for execution time measurement of the methods 
ids = [str(random.randint(100000, 9999999)) for _ in range(10000)]

test = Inventory('laptops.csv')

# time measurement for method with no dictionary
total_time_no_dict = 0
for identifier in ids:
    start = time.time()
    test.get_laptop_from_id(identifier)
    end = time.time()
    total_time_no_dict += (end - start)
    
# time measurement for method with dictionary
total_time_dict = 0
for identifier in ids:
    start = time.time()
    test.get_laptop_from_id_fast(identifier)
    end = time.time()
    total_time_dict += (end - start)
print(total_time_no_dict)
print(total_time_dict)

0.9696218967437744
0.004278898239135742


## ID Look-Up Performance Analysis of Both Methods
First method time completion was: 0.9696218967437744
Second updated time completion was: 0.004278898239135742

Dividing the first method by the second shows that improving the time complexity of the method caused the performance of the method to run 226 times faster.

# Step 6: Inventory Class, 3rd Update - Laptop Promotion Look-Up
 - Create method that checks if the customer can buy one or two laptops given their budget

In [6]:
# 3rd version of Inventory class - add check_promotion_dollars() method
# Method, given dollar amount, checks if can purchase up to two laptops

class Inventory():
    def __init__(self, csv_filename):
        
        # open file and convert to list of lists
        with open(csv_filename, encoding='utf8') as file:
            convert_file_to_list = list(csv.reader(file))
            self.header = convert_file_to_list[0]
            self.rows = convert_file_to_list[1:]
            
            # convert price column from string to integer
            for row in self.rows:
                row[12] = int(row[12])
        
        # pre-process data into dictionary
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
                
    # take a laptop id and return corresponding info if match found
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
    
    # new faster search method   
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        
    # method to check if can purchase up to two laptops, given dollar amount
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[12] == dollars:
                return True
        
        for row_1 in self.rows:
            for row_2 in self.rows:
                if (row_1[12] + row_2[12]) == dollars:
                    return True
        return False

True
False


## Testing the laptop promotion look-up method

In [ ]:
test = Inventory('laptops.csv')
print(test.check_promotion_dollars(1000))
print(test.check_promotion_dollars(442))

# Step 7: Inventory Class, 4th Update - Laptop Promotion Look-Up Method Time Complexity Improvement

In [7]:
# 4th version of Inventory class - add faster check_promotion_dollars method

class Inventory():
    def __init__(self, csv_filename):
        
        # open file and convert to list of lists
        with open(csv_filename, encoding='utf8') as file:
            convert_file_to_list = list(csv.reader(file))
            self.header = convert_file_to_list[0]
            self.rows = convert_file_to_list[1:]
            
            # convert price column from string to integer
            for row in self.rows:
                row[12] = int(row[12])
        
        # pre-process laptop ID data into dictionary
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
        # pre-process laptop pricing data into set
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[12])
                
    # take a laptop id and return corresponding info if match found
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
    
    # new faster search method   
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        
    # method to check if can purchase up to two laptops, given dollar amount
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[12] == dollars:
                return True
        
        for row_1 in self.rows:
            for row_2 in self.rows:
                if (row_1[12] + row_2[12]) == dollars:
                    return True
        return False
    
    # new faster search method with pre-processed pricing data
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price_1 in self.prices:
            for price_2 in self.prices:
                if (price_1 + price_2) == dollars:
                    return True
        return False

True
False


## Testing the improved laptop promotion look-up method

In [ ]:
test = Inventory('laptops.csv')

print(test.check_promotion_dollars_fast(1000))
print(test.check_promotion_dollars_fast(442))

# Step 8: Measure performance of the two laptop promotion look-up methods

In [29]:
# measure execution times of check_promotion_dollars and
# check_promotion_dollars_fast

# create list with 100 random values between 100 and 5,000
prices = [str(random.randint(100, 5000)) for _ in range(100)]
test = Inventory('laptops.csv')

# check execution time of check_promotion_dollars
total_time_no_set = 0
for price in prices:
    start = time.time()
    test.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += (end - start)
    
# check execution time of check_promotion_dollars_fast
total_time_set = 0
for price in prices:
    start = time.time()
    test.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += (end - start)

print(total_time_no_set)
print(total_time_set)

22.33216381072998
4.950788974761963


## Laptop Promotion Look-Up Performance Analysis of Both Methods
First method time completion was: 22.33216381072998
Second updated time completion was: 4.950788974761963

Dividing the first method by the second shows that improving the time complexity of the method caused the performance of the method to run 4.5 times faster.

# Final Step - Putting it All Together
# Step 9: Inventory Class, 5th Update - Finding Laptops within Customer Budget
1. add sorting of Inventory rows by laptop price when instantiating an Inventory class object
2. add binary search method
3. add another binary search method with improved time complexity

In [28]:
# 5th version of Inventory class
# help customer find all laptops given budget of D dollars
# in other words, write method to answer query: Given budget, find all laptops

# sort all laptops by price to then use binary search to find
# first laptop in the sorted list with a price larger than D

class Inventory():
    def __init__(self, csv_filename):
        
        # open file and convert to list of lists
        with open(csv_filename, encoding='utf8') as file:
            convert_file_to_list = list(csv.reader(file))
            self.header = convert_file_to_list[0]
            self.rows = convert_file_to_list[1:]
            
            # convert price column from string to integer
            for row in self.rows:
                row[12] = int(row[12])
        
        # pre-process laptop ID data into dictionary
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
        # pre-process laptop pricing data into set
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
            
        # sort self.rows by price
        self.rows_by_price = sorted(self.rows, key=row_price)
                
    # take a laptop id and return corresponding info if match found
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    # new faster search method   
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
        
    # method to check if can purchase up to two laptops, given dollar amount
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[12] == dollars:
                return True
        
        for row_1 in self.rows:
            for row_2 in self.rows:
                if (row_1[12] + row_2[12]) == dollars:
                    return True
        return False
    
    # new faster search method with pre-processed pricing data
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False
    
    # given binary search algorithm
    def find_laptop_with_price(self, target_price):
    range_start = 0                                   
    range_end = len(self.rows_by_price) - 1                       
    while range_start < range_end:
        range_middle = (range_end + range_start) // 2  
        price = self.rows_by_price[range_middle][-1]
        if price == target_price:                            
            return range_middle                        
        elif value < target_value:                           
            range_start = range_middle + 1             
        else:                                          
            range_end = range_middle - 1 
    price = self.rows_by_price[range_start][-1]
    if price != target_price:                  
        return -1                                      
    return range_start
    
    # altered binary search algorithm
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0
        range_end = len(self.rows_by_price) - 1 # subtracting by one is necessary because range begins at 0
        
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2
            price = self.rows_by_price[range_middle][-1]
            
            if price > target_price:
                range_end = range_middle
            
            else:
                range_start = range_middle + 1 # adding one is necessary to get just one higher index of target price index
        
        """if target price (budget) is higher than all laptop prices, range start will continuously
        increase until greater than range end, ending the while loop. range start will still be
        lower than target price, thus returning -1, meaning all laptops are within budget"""
        if self.rows_by_price[range_start][-1] <= target_price
            return -1
        
        return range_start

## Testing the laptops within budget look-up method

In [ ]:
test = Inventory('laptops.csv')

print(test.find_first_laptop_more_expensive(1000))
print(test.find_first_laptop_more_expensive(10000))

# Conclusion

In this project, three functionalities were created:
1. Looking up laptops by their ID number
2. Seeing how many laptops a customer could afford given their budget
3. Determining the highest priced laptop at which the customer could not afford; determining all of the laptops within the customer's budget

The three built-in Python modules used were: csv, time, and random

In this project, data pre-processing was used to significantly improve the time efficiency of the methods.